In [1]:
import pandas as pd
from zipfile import ZipFile
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from imblearn.over_sampling import SMOTE
from collections import Counter
import numpy as np

# Data Preparation (done from past weeks)

In [2]:
zip_file = ZipFile('../../data/raw/forStudents.pkl.zip')
df = pd.read_pickle(zip_file.open('forStudents.pkl'))
df

,acquisition_type,snapshot_type,channel,evaluation_dt,vintage,is_app_approved,vantage3_score,bad,bad_v2,bad_balance,...,rtr5930,rtr7110,rtr7140,rtr7160,rtr7216,rtr7620,rtr8120,rtr8220,rtr8320,rowNumber
0,ORGANIC,FLOW_MODULARITY,OTHER,2021-08-04,202108,1,698.0,0.0,0.0,0.0,...,2.0,4.0,4.0,9.0,5.0,100.0,5.0,5.0,5.0,59486
1,ORGANIC,FLOW_MODULARITY,OTHER,2021-03-01,202103,1,731.0,0.0,0.0,0.0,...,30.0,2.0,2.0,130.0,12.0,100.0,4.0,4.0,4.0,2357
2,ORGANIC,FLOW_MODULARITY,OTHER,2021-02-26,202102,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90398
3,ORGANIC,FLOW_MODULARITY,OTHER,2021-06-04,202106,1,715.0,0.0,0.0,0.0,...,249.0,18.0,18.0,56.0,7.0,100.0,5.0,151.0,45.0,104654
4,ORGANIC,FLOW_MODULARITY,OTHER,2021-07-14,202107,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48707,ORGANIC,FLOW_MODULARITY,OTHER,2021-05-01,202105,1,732.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,131881
48708,ORGANIC,FLOW_MODULARITY,OTHER,2020-11-29,202011,1,705.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,47197
48709,ORGANIC,FLOW_MODULARITY,OTHER,2021-04-25,202104,1,722.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,NaN,2.0,167.0,86.0,29739
48710,ORGANIC,FLOW_MODULARITY,OTHER,2021-04-10,202104,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89207


In [3]:
# Clean and imputation

df_clean = df[df['all0000'].notna()]
df_clean = df_clean[df_clean['bad'] != -1.0]
df_clean = df_clean[df_clean['is_app_approved'] == 1]
df_clean.drop(columns=['bad_v2','evaluation_dt','vintage', "vantage3_score", "bad_balance", "current_balance", "net_spend"], inplace=True)
described = df_clean["annual_income"].describe()
quartiles = list(described.iloc[4:7])

In [4]:
def assign_quartile(row):
    income = row["annual_income"]
    if income < quartiles[0]:
        return 0
    elif ((quartiles[0] < income) & (quartiles[1] >= income)):
        return 1
    elif ((quartiles[1] < income) & (quartiles[2] >= income)):
        return 2
    else:
        return 3

In [5]:
df_clean.apply(assign_quartile, axis=1).value_counts().sort_index()

0    10625
1    10546
2    10709
3    10667
dtype: int64

In [6]:
df_clean['income_quartile'] = df_clean.apply(assign_quartile, axis=1)

In [7]:
dfs = []
for i in range(4):
    quartile = df_clean.loc[df_clean["income_quartile"] == i]
    dfs.append(quartile.fillna(quartile.mean()))
    
df_clean = pd.concat(dfs)

temp = df_clean.isna().sum().to_frame()
temp[temp[0] != 0]

df_clean.drop(columns=['all9230', 'all9240', 'all9249', 'all9280'], inplace=True)
df_clean = df_clean.fillna(df_clean.mean())

/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_41925/448997779.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfs.append(quartile.fillna(quartile.mean()))
/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_41925/448997779.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_clean = df_clean.fillna(df_clean.mean())


In [8]:
# Categorical

def process_categorical_data(dataset):
    """ One hot encodes all of the categorial columns of the dataset.  Removes the original columns """
    # select categorical data
    categorical_data = dataset.select_dtypes('object')
    
    enc = OneHotEncoder(handle_unknown='ignore')
    
    ohe_data = enc.fit_transform(categorical_data).toarray()
    column_names = enc.get_feature_names(categorical_data.columns)
    
    add = dataset.select_dtypes(exclude='object')
    added = pd.DataFrame(ohe_data,columns=column_names).astype(int)
    
    add.reset_index(drop = True, inplace= True)
    added.reset_index(drop = True, inplace= True)
    X = pd.concat([add, 
               added], axis=1)
    print(X.shape)
    return X

df_clean = process_categorical_data(df_clean)
df_clean

/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(42547, 887)


,is_app_approved,bad,annual_income,all0000,all0060,all0061,all0101,all0200,all0201,all0206,...,state_code_SD,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY
0,1,0.0,15171,10.0,8.000000,8.000000,0.0,9.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,20000,3.0,4.165954,5.237061,0.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,7000,6.0,4.165954,5.237061,0.0,4.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,18000,3.0,1.000000,1.000000,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1,0.0,30000,6.0,4.165954,5.237061,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42542,1,0.0,125000,12.0,9.305257,11.139763,0.0,11.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42543,1,0.0,101000,60.0,21.000000,11.139763,0.0,43.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42544,1,0.0,145000,53.0,9.305257,11.139763,0.0,52.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42545,1,0.0,72000,7.0,9.305257,11.139763,0.0,7.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


# Week 4-6 starts here:

### Brainstorm with your team some ideas of new features you can create.

We are going to attempt to reduce the dimensionality of our features. PCA can remove correlated features, reduce overfitting, and thus improve the accuracy of our ML model.

In [9]:
df_clean

,is_app_approved,bad,annual_income,all0000,all0060,all0061,all0101,all0200,all0201,all0206,...,state_code_SD,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY
0,1,0.0,15171,10.0,8.000000,8.000000,0.0,9.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,20000,3.0,4.165954,5.237061,0.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,7000,6.0,4.165954,5.237061,0.0,4.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,18000,3.0,1.000000,1.000000,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1,0.0,30000,6.0,4.165954,5.237061,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42542,1,0.0,125000,12.0,9.305257,11.139763,0.0,11.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42543,1,0.0,101000,60.0,21.000000,11.139763,0.0,43.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42544,1,0.0,145000,53.0,9.305257,11.139763,0.0,52.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42545,1,0.0,72000,7.0,9.305257,11.139763,0.0,7.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


# Baseline model using underesamlping:

In [10]:
# balancing df clean using undersampling method
df_clean_0 = df_clean[df_clean['bad'] == 0.0]
df_clean_1 = df_clean[df_clean['bad'] == 1.0]

df_clean_0_sample = df_clean_0.sample(n=len(df_clean_1), replace=False, random_state = 42)
df_clean_balanced = pd.concat([df_clean_0_sample, df_clean_1])

len(df_clean_balanced) # should be 3693 * 2 = 7386

7386

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [12]:
def model_evaluator(data):
    X = data.drop('bad', axis=1)
    y = data['bad'].astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = LogisticRegression(penalty='none').fit(X_train,y_train)
    y_pred =clf.predict(X_test)
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    X_test['predicted_bad'] = y_pred
    return clf
#     print("Dollar Loss: " + str(X_test[(y_test == 1) & (y_test == y_pred)]['bad_balance'].sum().round(2)))

In [13]:
clf = model_evaluator(df_clean_balanced) # BASELINE MODEL HERE

[[510 604]
 [319 783]]
              precision    recall  f1-score   support

           0       0.62      0.46      0.52      1114
           1       0.56      0.71      0.63      1102

    accuracy                           0.58      2216
   macro avg       0.59      0.58      0.58      2216
weighted avg       0.59      0.58      0.58      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)

In [16]:
scaled_data

,is_app_approved,annual_income,all0000,all0060,all0061,all0101,all0200,all0201,all0206,all0217,...,state_code_SD,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY
0,0.0,-0.063132,-0.271052,-0.450030,-0.565297,-0.094785,-0.392699,-0.02716,-0.22507,-0.203263,...,-0.04034,-0.1328,4.098486,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
1,0.0,-0.057225,-0.723091,-0.450030,-0.565297,-0.094785,-0.682158,-0.02716,-0.22507,-0.607925,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
2,0.0,-0.026283,-0.271052,-1.298225,0.233685,-0.094785,-0.199727,-0.02716,-0.22507,0.066512,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
3,0.0,-0.026521,0.271395,-0.483818,0.233685,-0.094785,0.282705,-0.02716,-0.22507,0.740950,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
4,0.0,-0.034613,-0.451868,-0.280217,-0.616574,-0.094785,-0.392699,-0.02716,-0.22507,-0.338150,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,0.0,0.027272,0.361803,1.144995,0.897553,-0.094785,0.379192,-0.02716,-0.22507,0.201400,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7382,0.0,-0.002480,0.542618,0.126987,-0.183967,-0.094785,-0.585672,-0.02716,-0.22507,-0.607925,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7383,0.0,0.033223,1.265881,0.534190,0.248641,-0.094785,1.440542,-0.02716,-0.22507,0.606062,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7384,0.0,-0.009621,0.180987,-0.076615,-0.400271,-0.094785,0.282705,-0.02716,-0.22507,0.336287,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034


# Principal Component Analysis

In [17]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
pca = PCA(n_components=50)
pca.fit(scaled_data)
transformed_data = pca.fit_transform(scaled_data)
pca_data = pd.DataFrame(transformed_data)

In [18]:
pca_data

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-7.177256,-4.512930,4.265165,2.081165,-2.684958,-1.721772,-2.413139,4.802513,-0.447673,-1.828041,...,0.519037,-0.574599,0.602711,0.402628,-1.523708,0.900033,0.555136,-0.541026,1.298331,-1.259420
1,-8.489437,-0.076108,4.600491,1.496669,1.943937,-1.350313,-0.997756,3.757003,-1.257610,-3.831411,...,0.230884,1.079226,0.191290,-0.469430,1.706147,0.331814,0.192654,1.150760,-0.199146,0.032113
2,-4.528945,2.689593,-1.525833,3.402562,-10.939203,32.022136,1.318510,4.352340,-2.980111,-0.087524,...,2.912176,-3.607516,5.887018,-0.053293,0.250252,-9.168252,-8.144587,3.653882,1.987360,-0.003131
3,8.412768,13.358951,-0.496733,0.207961,2.105100,0.205440,3.957855,-4.745028,-5.984150,-5.472662,...,-1.859254,2.420804,0.556076,1.633248,-0.575393,-3.924366,0.113902,3.055655,0.501336,-2.222453
4,-3.332479,6.545506,1.212955,-3.897411,-0.315444,0.101071,-5.376314,-5.889101,-0.887421,-4.343398,...,1.514986,-1.844652,0.323513,-0.943879,-0.662326,-0.093116,-1.237746,-0.026872,1.005778,0.226680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,8.550572,-4.179188,0.556905,-4.050598,13.125279,0.769347,-1.940285,3.237748,-2.062003,1.792799,...,0.005906,2.386376,-4.815442,-1.323927,-2.013567,-2.423169,-0.754970,-1.985157,1.165206,1.126995
7382,-0.361224,-1.344797,-5.767633,-4.212039,-3.160671,-1.235665,-1.471947,0.741610,0.056708,1.521022,...,-1.434256,-2.897741,-2.063534,-1.550165,0.874856,-4.677651,0.774153,2.665515,1.988603,1.183768
7383,8.440756,12.255782,-7.983140,-3.144338,-4.322581,-4.804490,0.112822,3.336196,-5.525666,-1.816158,...,-1.145057,1.613742,1.039578,-0.119130,-0.267843,1.810377,1.873469,-1.566188,0.917395,-0.261429
7384,8.333123,-1.642463,-5.931626,-3.075135,6.921080,2.412139,-1.667400,-0.803670,0.836668,4.359491,...,0.230140,0.477795,-0.616092,1.039048,-0.748046,4.827508,-1.185688,0.474397,2.764735,2.073527


In [19]:
def pca_model(new_x, new_y):
    X_train, X_test, y_train, y_test = train_test_split(new_x, new_y, test_size=0.3, random_state=42)

    clf = LogisticRegression(penalty='none').fit(X_train,y_train)
#     clf = RandomForestClassifier(max_depth = 15, random_state=42).fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    X_test['predicted_bad'] = y_pred
#     print("Dollar Loss: " + str(X_test[(y_test == 1) & (y_test == y_pred)]['bad_balance'].sum().round(2)))

In [20]:
pca_model(pca_data, y)

[[682 432]
 [387 715]]
              precision    recall  f1-score   support

           0       0.64      0.61      0.62      1114
           1       0.62      0.65      0.64      1102

    accuracy                           0.63      2216
   macro avg       0.63      0.63      0.63      2216
weighted avg       0.63      0.63      0.63      2216



In [21]:
# accuracy went from 0.58 to 0.63 after pca! nice! also, f1-score for the 0 class improved.

In [22]:
# TODO: balance df_clean
df_clean_0 = df_clean[df_clean['bad'] == 0.0]
df_clean_1 = df_clean[df_clean['bad'] == 1.0]

df_clean_0_sample = df_clean_0.sample(n=len(df_clean_1), replace=False, random_state = 42)
df_clean_balanced = pd.concat([df_clean_0_sample, df_clean_1])

len(df_clean_balanced) # should be 3693 * 2 = 7386



7386

# Top 50 features and top 10 features

In [23]:
X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

In [24]:
# X = X.drop('vantage3_score', axis=1).drop('bad_balance', axis=1).drop('is_app_approved',axis=1)

In [25]:
top_col_idx = abs(clf.coef_[0]).argsort()[-50:]
top_col_idx

array([554, 646, 303, 465, 498, 508, 144, 166, 165, 716, 145, 304, 521,
       805, 642, 509, 202, 468, 441, 718, 466, 810, 809, 438, 160, 693,
       520, 548, 274, 161, 311, 644, 690, 328, 640, 691, 649, 162, 676,
       717, 697, 715, 714, 272, 167, 692, 757, 648, 271, 694])

In [26]:
def model_evaluator(data):
    X = data.drop('bad', axis=1)
    y = data['bad'].astype(int)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

    clf = LogisticRegression(penalty='none').fit(X_train,y_train)
    y_pred =clf.predict(X_test)
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    X_test['predicted_bad'] = y_pred
#     print("Dollar Loss: " + str(X_test[(y_test == 1) & (y_test == y_pred)]['bad_balance'].sum().round(2)))

In [27]:
top50 = X.iloc[:,top_col_idx]
top50['bad'] = y

/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_41925/130046108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top50['bad'] = y


In [28]:
# top50 = X.iloc[:,scores.argsort()[:1]]
# top50['bad'] = y


In [29]:
model_evaluator(top50)

[[585 529]
 [388 714]]
              precision    recall  f1-score   support

           0       0.60      0.53      0.56      1114
           1       0.57      0.65      0.61      1102

    accuracy                           0.59      2216
   macro avg       0.59      0.59      0.58      2216
weighted avg       0.59      0.59      0.58      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## chi-sq select k best

In [30]:
from sklearn.feature_selection import SelectKBest, chi2

In [31]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_normalized = scaler.fit_transform(X)

In [32]:
X_new = pd.DataFrame(SelectKBest(chi2, k=50).fit_transform(X_normalized, y))
X_new['bad'] = y

model_evaluator(top50)

[[585 529]
 [388 714]]
              precision    recall  f1-score   support

           0       0.60      0.53      0.56      1114
           1       0.57      0.65      0.61      1102

    accuracy                           0.59      2216
   macro avg       0.59      0.59      0.58      2216
weighted avg       0.59      0.59      0.58      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


What are your top 50 features?  Top 10 features?

In [33]:
# Top 50 features with feature importance

X.iloc[:, top_col_idx]

,iln5220,bcc5422,mta5020,stu5123,rec5120,fip5020,all5020,all5830,all5825,reh5120,...,brc5930,reh5030,reh5020,use5320,all5935,brc5038,rev5036,bcc5520,use5030,brc5620
6754,325.000000,1700.000000,76625.021127,7926.483675,10429.937500,2568.024286,15294.0,385.0,385.0,669.0,...,249.0,778.0,778.0,10793.749338,574.0,109.000000,451.636409,2383.00000,2108.783641,13522.0
1983,176.017635,3800.000000,76625.021127,7926.483675,10429.937500,2568.024286,809.0,108.0,108.0,374.0,...,28.0,789.0,789.0,10793.749338,48.0,51.000000,451.636409,1789.00000,2108.783641,4577.0
25807,1.000000,700.000000,141414.140673,14800.060567,13823.402985,10.000000,474.0,61.0,61.0,236.0,...,231.0,464.0,464.0,8201.291358,232.0,0.000000,464.000000,500.00000,1992.075506,6236.0
25211,745.000000,500.000000,141414.140673,14800.060567,13823.402985,4620.372275,41179.0,912.0,912.0,722.0,...,430.0,2090.0,2090.0,8201.291358,1175.0,1017.000000,884.985469,550.00000,1992.075506,1210.0
23604,25.000000,500.000000,141414.140673,14800.060567,13823.402985,4620.372275,630.0,169.0,169.0,344.0,...,0.0,344.0,344.0,8201.291358,144.0,1988.472697,884.985469,400.00000,1992.075506,856.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,469.928300,2500.000000,210748.620051,22617.714663,18401.614815,6415.659231,16164.0,363.0,363.0,9746.0,...,200.0,16164.0,16164.0,10550.587071,200.0,9746.000000,642.000000,4175.00000,2480.986044,1024.0
42490,384.000000,2328.240767,210748.620051,9564.000000,18401.614815,380.000000,20201.0,427.0,427.0,0.0,...,0.0,0.0,0.0,10550.587071,434.0,2793.698409,0.000000,2147.05802,2480.986044,669.0
42494,600.000000,500.000000,210748.620051,22617.714663,18401.614815,13166.000000,34576.0,1398.0,1398.0,3.0,...,602.0,3.0,3.0,10550.587071,3078.0,3.000000,0.000000,375.00000,2480.986044,2027.0
42520,570.000000,500.000000,210748.620051,22617.714663,18401.614815,6415.659231,46134.0,1323.0,1323.0,6788.0,...,2588.0,8480.0,8480.0,10550.587071,3628.0,7675.000000,1219.406468,2613.00000,2480.986044,1970.0


In [34]:
# Top 10 features
X.iloc[:, top_col_idx[-10:]]

,brc5930,reh5030,reh5020,use5320,all5935,brc5038,rev5036,bcc5520,use5030,brc5620
6754,249.0,778.0,778.0,10793.749338,574.0,109.000000,451.636409,2383.00000,2108.783641,13522.0
1983,28.0,789.0,789.0,10793.749338,48.0,51.000000,451.636409,1789.00000,2108.783641,4577.0
25807,231.0,464.0,464.0,8201.291358,232.0,0.000000,464.000000,500.00000,1992.075506,6236.0
25211,430.0,2090.0,2090.0,8201.291358,1175.0,1017.000000,884.985469,550.00000,1992.075506,1210.0
23604,0.0,344.0,344.0,8201.291358,144.0,1988.472697,884.985469,400.00000,1992.075506,856.0
...,...,...,...,...,...,...,...,...,...,...
42477,200.0,16164.0,16164.0,10550.587071,200.0,9746.000000,642.000000,4175.00000,2480.986044,1024.0
42490,0.0,0.0,0.0,10550.587071,434.0,2793.698409,0.000000,2147.05802,2480.986044,669.0
42494,602.0,3.0,3.0,10550.587071,3078.0,3.000000,0.000000,375.00000,2480.986044,2027.0
42520,2588.0,8480.0,8480.0,10550.587071,3628.0,7675.000000,1219.406468,2613.00000,2480.986044,1970.0


In [35]:
model_evaluator(top50)

[[585 529]
 [388 714]]
              precision    recall  f1-score   support

           0       0.60      0.53      0.56      1114
           1       0.57      0.65      0.61      1102

    accuracy                           0.59      2216
   macro avg       0.59      0.59      0.58      2216
weighted avg       0.59      0.59      0.58      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
top10 = X.iloc[:,top_col_idx[-10:]]
top10['bad'] = y

/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_41925/4056120755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10['bad'] = y


In [37]:
top10

,brc5930,reh5030,reh5020,use5320,all5935,brc5038,rev5036,bcc5520,use5030,brc5620,bad
6754,249.0,778.0,778.0,10793.749338,574.0,109.000000,451.636409,2383.00000,2108.783641,13522.0,0
1983,28.0,789.0,789.0,10793.749338,48.0,51.000000,451.636409,1789.00000,2108.783641,4577.0,0
25807,231.0,464.0,464.0,8201.291358,232.0,0.000000,464.000000,500.00000,1992.075506,6236.0,0
25211,430.0,2090.0,2090.0,8201.291358,1175.0,1017.000000,884.985469,550.00000,1992.075506,1210.0,0
23604,0.0,344.0,344.0,8201.291358,144.0,1988.472697,884.985469,400.00000,1992.075506,856.0,0
...,...,...,...,...,...,...,...,...,...,...,...
42477,200.0,16164.0,16164.0,10550.587071,200.0,9746.000000,642.000000,4175.00000,2480.986044,1024.0,1
42490,0.0,0.0,0.0,10550.587071,434.0,2793.698409,0.000000,2147.05802,2480.986044,669.0,1
42494,602.0,3.0,3.0,10550.587071,3078.0,3.000000,0.000000,375.00000,2480.986044,2027.0,1
42520,2588.0,8480.0,8480.0,10550.587071,3628.0,7675.000000,1219.406468,2613.00000,2480.986044,1970.0,1


In [38]:
model_evaluator(top10)

[[505 609]
 [323 779]]
              precision    recall  f1-score   support

           0       0.61      0.45      0.52      1114
           1       0.56      0.71      0.63      1102

    accuracy                           0.58      2216
   macro avg       0.59      0.58      0.57      2216
weighted avg       0.59      0.58      0.57      2216



Features we dropped include dates, and features like bad_balance and vantage3_score which have direct relation to our outcome variable bad

# Week 6-7 starts here:

### PLAN:
#### 1) Using downsampling (same as last week) test XGBoost and RFC, compare accuracy to LogisticRegression model
#### 2) Repeat steps above using upsampling.

In [39]:
# ! pip install xgboost

In [40]:
# STEP 1.0: Downsampling LogisticRegression

X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[693 421]
 [375 727]]
              precision    recall  f1-score   support

           0       0.65      0.62      0.64      1114
           1       0.63      0.66      0.65      1102

    accuracy                           0.64      2216
   macro avg       0.64      0.64      0.64      2216
weighted avg       0.64      0.64      0.64      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [41]:
# STEP 1.1: Downsampling XGBoost

from xgboost import XGBClassifier

X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[680 434]
 [377 725]]
              precision    recall  f1-score   support

           0       0.64      0.61      0.63      1114
           1       0.63      0.66      0.64      1102

    accuracy                           0.63      2216
   macro avg       0.63      0.63      0.63      2216
weighted avg       0.63      0.63      0.63      2216



In [42]:
# STEP 1.2: Downsampling RFC

from xgboost import XGBClassifier

X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[666 448]
 [358 744]]
              precision    recall  f1-score   support

           0       0.65      0.60      0.62      1114
           1       0.62      0.68      0.65      1102

    accuracy                           0.64      2216
   macro avg       0.64      0.64      0.64      2216
weighted avg       0.64      0.64      0.64      2216



In [43]:
# STEP 2.0: Upsampling Logistic Regression

# first do the upsampling
from imblearn.over_sampling import SMOTE
from collections import Counter

oversample = SMOTE()
X = df_clean.drop(columns='bad')
y = df_clean.bad.astype(int)
counter = Counter(y)
print(counter)

X,y = oversample.fit_resample(X,y)
counter = Counter(y)
print(counter)

# second, standard scale the data
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

# third, train test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Counter({0: 38854, 1: 3693})
Counter({0: 38854, 1: 38854})
[[11595   113]
 [ 1178 10427]]
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     11708
           1       0.99      0.90      0.94     11605

    accuracy                           0.94     23313
   macro avg       0.95      0.94      0.94     23313
weighted avg       0.95      0.94      0.94     23313



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
# STEP 2.1: Upsampling XGBoost

# first do the upsampling
from imblearn.over_sampling import SMOTE
from collections import Counter

oversample = SMOTE()
X = df_clean.drop(columns='bad')
y = df_clean.bad.astype(int)
counter = Counter(y)
print(counter)

X,y = oversample.fit_resample(X,y)
counter = Counter(y)
print(counter)

# second, standard scale the data
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

# third, train test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Counter({0: 38854, 1: 3693})
Counter({0: 38854, 1: 38854})
[[11626    82]
 [ 1055 10550]]
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     11708
           1       0.99      0.91      0.95     11605

    accuracy                           0.95     23313
   macro avg       0.95      0.95      0.95     23313
weighted avg       0.95      0.95      0.95     23313



In [45]:
# STEP 2.2: Upsampling RFC

# first do the upsampling
from imblearn.over_sampling import SMOTE
from collections import Counter

oversample = SMOTE()
X = df_clean.drop(columns='bad')
y = df_clean.bad.astype(int)
counter = Counter(y)
print(counter)

X,y = oversample.fit_resample(X,y)
counter = Counter(y)
print(counter)

# second, standard scale the data
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X), columns = X.columns)
X = scaled_data

# third, train test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Counter({0: 38854, 1: 3693})
Counter({0: 38854, 1: 38854})
[[11675    33]
 [ 1048 10557]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     11708
           1       1.00      0.91      0.95     11605

    accuracy                           0.95     23313
   macro avg       0.96      0.95      0.95     23313
weighted avg       0.96      0.95      0.95     23313



##### Evaluate the change in performance between logistic regression and new model.

We tried a total of 3 different models and 2 different sampling techniques for a total of 6 techniques. The accuracies are listed below.

Downsampling Logistic Regression: 0.64

Downsampling XGBoost: 0.63

Downsampling Random Forest: 0.64

Upsampling Logistic Regression: 0.94

Upsampling XGBoost: 0.95

Upsampling Random Forest: 0.95

In conclusion, our best combination of sampling technique and modeling is when we upsample and use an Random Forest Classifier.


##### Discuss different techniques you can apply, pros and cons. Why did you choose this technique?

We tried two different sampling techniques: upsampling and downsampling on all 3 of our models. Downsampling is not preferred because we lose a lot of valuable data which doesn’t occur with upsampling. At the same time, upsampling creates synthetic data that may also negatively affect our model. 

The three models we are using are: logistic regression (baseline), random forest tree, and XGBoost. Logistic regression is fast, common, reliable, and interpretable yet may not be as powerful as the others. Random forest tree is an alternative approach to logistic regression and can be more beneficial depending on the data. XGBoost finally is one of the most optimal algorithms and is extremely powerful, though it is more computationally expensive and takes a long time to run! In our final model selection, we will have to look at all of these pros and cons and decide what aspects of the model we want to emphasize/focus on and select accordingly.